In [22]:
#Get the model
#Use paths to determine which type of classification

#path_result='C:\\Backup of covid project\\2cls_posVSnonpos\\results\\'
#path_data="C:\\Backup of covid project\\2cls_posVSnonpos\\data\\"
#path_code="C:\\Backup of covid project\\"

path_result='C:\\Backup of covid project\\2cls_negVSnonneg\\results\\'
path_data="C:\\Backup of covid project\\2cls_negVSnonneg\\data\\"
path_code="C:\\Backup of covid project\\"

#path_result='C:\\Backup of covid project\\2cls_wlNeu_woNeu\\results_WOneu\\'
#path_data="C:\\Backup of covid project\\2cls_wlNeu_woNeu\\data\\"
#path_code="C:\\Backup of covid project\\"

import time
import numpy as np
import tensorflow as tf
import os
import random
os.chdir(path_code)
import gat
import imp
imp.reload(gat)
from gat import GAT, HeteGAT, HeteGAT_multi  # or * for that matter

import process
import importlib
importlib.reload(process)
# 禁用gpu
import numpy as np


os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

dataset = 'acm'
featype = 'fea'
checkpt_file=path_result+'acm_allMP_multi_fea_.ckpt'
#checkpt_file = 'C:\data\HAN-modified\\tranfer learning\\acm_allMP_multi_fea_2cls.ckpt'

print('model: {}'.format(checkpt_file))
# training params for only CC
batch_size = 1
nb_epochs = 100
patience = 100
lr = 0.02  # learning rate
l2_coef = 0.08#0.1#0.001  # weight decay
# numbers of hidden units per each attention head in each layer
hid_units = [8]
n_heads = [4, 1]  # additional entry for the output layer
residual = False
nonlinearity = tf.nn.elu
model = HeteGAT_multi

print('Dataset: ' + dataset)
print('----- Opt. hyperparams -----')
print('lr: ' + str(lr))
print('l2_coef: ' + str(l2_coef))
print('----- Archi. hyperparams -----')
print('nb. layers: ' + str(len(hid_units)))
print('nb. units per layer: ' + str(hid_units))
print('nb. attention heads: ' + str(n_heads))
print('residual: ' + str(residual))
print('nonlinearity: ' + str(nonlinearity))
print('model: ' + str(model))

# jhy data
import scipy.io as sio
import scipy.sparse as sp


def sample_mask(idx, l):
    """Create mask."""
    mask = np.zeros(l)
    mask[idx] = 1
    return np.array(mask, dtype=np.bool)

def load_data_dblp(path=path_data):
    import pandas as pd
    import pickle
    import scipy.sparse
    from scipy.sparse import csc_matrix
    from scipy import sparse
    import numpy as np 
    import os 
    import csv 
    import numpy as np
    schools = ["notredame","uofm","columbia","dartmouth","UCSD","berkeley","Harvard","ucla"]
    s=schools[3]#use this index to control which network we are going to run.
    t="comment"
    
    os.chdir(path)
    name="label_emt3_"+s+"_cm.csv"
    truelabels=pd.read_csv(name,skip_blank_lines=True,header=None)
    name="feature_"+s+t+".csv"
    truefeatures=pd.read_csv(name,skip_blank_lines=True,header=None).values

    N=truefeatures.shape[0]
    name="CCsym_"+s+".npz"
    dat_cc=scipy.sparse.load_npz(name);dat_cc=dat_cc.toarray();
    #name="CCasym_"+s+".npz"
    #dat_cca=scipy.sparse.load_npz(name);dat_cca=dat_cca.toarray();
    name="CSCsy_"+s+".npz"
    dat_csc=scipy.sparse.load_npz(name);dat_csc=dat_csc.toarray();
    name="CACsy_"+s+".npz"
    dat_cac=scipy.sparse.load_npz(name);dat_cac=dat_cac.toarray();
    #rownetworks = [dat_csc,dat_cac]#,dat_cc]
    #rownetworks = [np.transpose(dat_cc)]
    rownetworks = [(dat_cc)]
    #rownetworks = [np.zeros((N,N))]

    y=truelabels
    name="train_index_"+s+"_cm.p"
    train_idx = pickle.load(open(name,"rb"));train_idx=np.array(train_idx)
    name="to_be_labeled_index_"+s+"_cm.p"
    val_idx = pickle.load(open(name,"rb"));val_idx=np.array(val_idx)
    name="test_index_"+s+"_cm.p"
    test_idx = pickle.load(open(name,"rb"));test_idx=np.array(test_idx)

    train_mask = sample_mask(train_idx, y.shape[0])
    val_mask = sample_mask(val_idx, y.shape[0])
    test_mask = sample_mask(test_idx, y.shape[0])
    
    y_train = np.zeros(y.shape)
    y_val = np.zeros(y.shape)
    y_test = np.zeros(y.shape)
    y=y.values
    y_train[train_mask, :] = y[train_mask, :]
    y_val[val_mask, :] = y[val_mask, :]
    y_test[test_mask, :] = y[test_mask, :]

    # return selected_idx, selected_idx_2
    print('y_train:{}, y_val:{}, y_test:{}, train_idx:{}, val_idx:{}, test_idx:{}'.format(y_train.shape,
                                                                                          y_val.shape,
                                                                                          y_test.shape,
                                                                                          train_idx.shape,
                                                                                          val_idx.shape,
                                                                                          test_idx.shape))
    truefeatures_list = [truefeatures, truefeatures]#, truefeatures, truefeatures]# ?? why copy three times? First for center node, 2 for each metapath.    
    os.chdir(path_code)
    return rownetworks, truefeatures_list, y_train, y_val, y_test, train_mask, val_mask, test_mask





# use adj_list as fea_list, have a try~
adj_list, fea_list, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data_dblp()
if featype == 'adj':
    fea_list = adj_list



import scipy.sparse as sp

nb_nodes = fea_list[0].shape[0]
ft_size = fea_list[0].shape[1]
nb_classes = y_train.shape[1]

# features = features[np.newaxis]  # [1, nb_node, ft_size]
fea_list = [fea[np.newaxis] for fea in fea_list]
adj_list = [adj[np.newaxis] for adj in adj_list]
y_train = y_train[np.newaxis]
y_val = y_val[np.newaxis]
y_test = y_test[np.newaxis]
train_mask = train_mask[np.newaxis]
val_mask = val_mask[np.newaxis]
test_mask = test_mask[np.newaxis]

biases_list = [process.adj_to_bias(adj, [nb_nodes], nhood=1) for adj in adj_list]
#biases_list = [process.adj_to_bias(adj) for adj in adj_list]
#process.adj_to_bias(adj)
print('build graph...')
with tf.Graph().as_default():
    with tf.name_scope('input'):
        ftr_in_list = [tf.placeholder(dtype=tf.float32,
                                      shape=(batch_size, nb_nodes, ft_size),
                                      name='ftr_in_{}'.format(i))
                       for i in range(len(fea_list))]
        bias_in_list = [tf.placeholder(dtype=tf.float32,
                                       shape=(batch_size, nb_nodes, nb_nodes),
                                       name='bias_in_{}'.format(i))
                        for i in range(len(biases_list))]
        lbl_in = tf.placeholder(dtype=tf.int32, shape=(
            batch_size, nb_nodes, nb_classes), name='lbl_in')
        msk_in = tf.placeholder(dtype=tf.int32, shape=(batch_size, nb_nodes),
                                name='msk_in')
        attn_drop = tf.placeholder(dtype=tf.float32, shape=(), name='attn_drop')
        ffd_drop = tf.placeholder(dtype=tf.float32, shape=(), name='ffd_drop')
        is_train = tf.placeholder(dtype=tf.bool, shape=(), name='is_train')
    # forward
    logits, final_embedding, att_val = model.inference(ftr_in_list, nb_classes, nb_nodes, is_train,
                                                       attn_drop, ffd_drop,
                                                       bias_mat_list=bias_in_list,
                                                       hid_units=hid_units, n_heads=n_heads,
                                                       residual=residual, activation=nonlinearity)

    # cal masked_loss
    log_resh = tf.reshape(logits, [-1, nb_classes])
    lab_resh = tf.reshape(lbl_in, [-1, nb_classes])
    msk_resh = tf.reshape(msk_in, [-1])
    loss = model.masked_softmax_cross_entropy(log_resh, lab_resh, msk_resh)
    predicted=tf.argmax(log_resh, 1)
    accuracy = model.masked_accuracy(log_resh, lab_resh, msk_resh)
    # optimzie
    train_op = model.training(loss, lr, l2_coef)

    saver = tf.train.Saver()

    init_op = tf.group(tf.global_variables_initializer(),
                       tf.local_variables_initializer())

    vlss_mn = np.inf
    vacc_mx = 0.0
    curr_step = 0

    with tf.Session(config=config) as sess:
        sess.run(init_op)

        train_loss_avg = 0
        train_acc_avg = 0
        val_loss_avg = 0
        val_acc_avg = 0
        val_pred_avg = 0
        for epoch in range(nb_epochs):
            tr_step = 0
           
            tr_size = fea_list[0].shape[0]
            # ================   training    ============
            while tr_step * batch_size < tr_size:

                fd1 = {i: d[tr_step * batch_size:(tr_step + 1) * batch_size]
                       for i, d in zip(ftr_in_list, fea_list)}
                fd2 = {i: d[tr_step * batch_size:(tr_step + 1) * batch_size]
                       for i, d in zip(bias_in_list, biases_list)}
                fd3 = {lbl_in: y_train[tr_step * batch_size:(tr_step + 1) * batch_size],
                       msk_in: train_mask[tr_step * batch_size:(tr_step + 1) * batch_size],
                       is_train: True,
                       attn_drop: 0,
                       ffd_drop: 0}
                fd = fd1
                fd.update(fd2)
                fd.update(fd3)
                _, loss_value_tr, acc_tr, att_val_train = sess.run([train_op, loss, accuracy, att_val],
                                                                   feed_dict=fd)
                train_loss_avg += loss_value_tr
                train_acc_avg += acc_tr
                tr_step += 1

            vl_step = 0
            vl_size = fea_list[0].shape[0]

            # import pdb; pdb.set_trace()
            print('Epoch: {}, att_val: {}'.format(epoch, np.mean(att_val_train, axis=0)))
            print('Training: loss = %.5f, acc = %.5f |' %
                  (train_loss_avg / tr_step, train_acc_avg / tr_step))

            curr_step += 1
            if epoch==nb_epochs-1:
                saver.save(sess, checkpt_file)

            train_loss_avg = 0
            train_acc_avg = 0
            val_loss_avg = 0
            val_acc_avg = 0
        #save_path = saver.save(sess, "C:\data\HAN-modified\\tranfer learning\model.ckpt")
        #print("Model saved in path: %s" % save_path)
        saver.restore(sess, checkpt_file)
        print('load model from : {}'.format(checkpt_file))
        ts_size = fea_list[0].shape[0]
        ts_step = 0
        ts_loss = 0.0
        ts_acc = 0.0

        while ts_step * batch_size < ts_size:
            # fd1 = {ftr_in: features[ts_step * batch_size:(ts_step + 1) * batch_size]}
            fd1 = {i: d[ts_step * batch_size:(ts_step + 1) * batch_size]
                   for i, d in zip(ftr_in_list, fea_list)}
            fd2 = {i: d[ts_step * batch_size:(ts_step + 1) * batch_size]
                   for i, d in zip(bias_in_list, biases_list)}
            fd3 = {lbl_in: y_test[ts_step * batch_size:(ts_step + 1) * batch_size],
                   msk_in: test_mask[ts_step * batch_size:(ts_step + 1) * batch_size],
            
                   is_train: False,
                   attn_drop: 0.0,
                   ffd_drop: 0.0}
        
            fd = fd1
            fd.update(fd2)
            fd.update(fd3)
            loss_value_ts, acc_ts, jhy_final_embedding = sess.run([loss, accuracy, final_embedding],
                                                                  feed_dict=fd)
            ts_loss += loss_value_ts
            ts_acc += acc_ts
            ts_step += 1

        print('Test loss:', ts_loss / ts_step,
              '; Test accuracy:', ts_acc / ts_step)
        
        import pickle
        
        print('start knn, kmean.....')
        #os.chdir('C:\\data\\HAN-modified\\')
        xx = np.expand_dims(jhy_final_embedding, axis=0)[test_mask]

        from numpy import linalg as LA

        # xx = xx / LA.norm(xx, axis=1)
        yy = y_test[test_mask]

        print('xx: {}, yy: {}'.format(xx.shape, yy.shape))
        from jhyexp import my_KNN, my_Kmeans#, my_TSNE, my_Linear

        my_KNN(xx, yy)
        my_Kmeans(xx, yy)

        sess.close()


model: C:\Backup of covid project\2cls_negVSnonneg\results\acm_allMP_multi_fea_.ckpt
Dataset: acm
----- Opt. hyperparams -----
lr: 0.02
l2_coef: 0.08
----- Archi. hyperparams -----
nb. layers: 1
nb. units per layer: [8]
nb. attention heads: [4, 1]
residual: False
nonlinearity: <function elu at 0x000001A1544A9E58>
model: <class 'gat.HeteGAT_multi'>
y_train:(2009, 2), y_val:(2009, 2), y_test:(2009, 2), train_idx:(417,), val_idx:(1492,), test_idx:(100,)
build graph...
de
Epoch: 0, att_val: [1.]
Training: loss = 1.29802, acc = 0.61151 |
Epoch: 1, att_val: [1.]
Training: loss = 2.54631, acc = 0.70983 |
Epoch: 2, att_val: [1.]
Training: loss = 2.27496, acc = 0.76978 |
Epoch: 3, att_val: [1.]
Training: loss = 1.81248, acc = 0.81295 |
Epoch: 4, att_val: [1.]
Training: loss = 1.34823, acc = 0.83213 |
Epoch: 5, att_val: [1.]
Training: loss = 0.94134, acc = 0.83933 |
Epoch: 6, att_val: [1.]
Training: loss = 0.62805, acc = 0.84413 |
Epoch: 7, att_val: [1.]
Training: loss = 0.42898, acc = 0.82734 |

In [23]:
#Use model from above running to get predictions for all datasets
#the model used and type of classification depends on paths from previous block
#don't run this without running the previous one
#The print-outs in this block is useless and meaningless, 
#the correct prediction for each sample has been saved to paths specified in previous block

import time
import numpy as np
import tensorflow as tf
import os
os.chdir(path_code)
from gat import GAT, HeteGAT, HeteGAT_multi
import process
import importlib
from layers import attn_head, SimpleAttLayer
importlib.reload(process)
# 禁用gpu


os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

dataset = 'acm'
featype = 'fea'
# training params



print('Dataset: ' + dataset)
print('----- Opt. hyperparams -----')
#print('lr: ' + str(lr))
#print('l2_coef: ' + str(l2_coef))
#print('----- Archi. hyperparams -----')
#print('nb. layers: ' + str(len(hid_units)))
#print('nb. units per layer: ' + str(hid_units))
#print('nb. attention heads: ' + str(n_heads))
#print('residual: ' + str(residual))
#print('nonlinearity: ' + str(nonlinearity))
#print('model: ' + str(model))

# jhy data
import scipy.io as sio
import scipy.sparse as sp


def sample_mask(idx, l):
    """Create mask."""
    mask = np.zeros(l)
    mask[idx] = 1
    return np.array(mask, dtype=np.bool)

#path_cur='C:\\data\\HAN-modified\\tranfer learning\\'
#path_data='C:\\data\\HAN-modified\\data\\preprocessed-trial\\'

schools = ["notredame2019","notredame2020","uofm2019","uofm2020","columbia2019","columbia2020","dartmouth","UCSD2019","UCSD2020","berkeley2019","berkeley2020","Harvard2019","Harvard2020","ucla2019","ucla2020"]
schools_type = ["notredamecomment2019","notredamecomment2020","uofmcomment2019","uofmcomment2020","columbiacomment2019","columbiacomment2020","dartmouthcomment","UCSDcomment2019","UCSDcomment2020",
           "berkeleycomment2019","berkeleycomment2020","Harvardcomment2019","Harvardcomment2020","uclacomment2019","uclacomment2020"]


#path_data='C:\\data\\HAN-modified\\tranfer learning\\'
def load_data_dblp(s,st,path=path_data):
    import pandas as pd
    import pickle
    import scipy.sparse
    from scipy.sparse import csc_matrix
    from scipy import sparse
    import numpy as np 
    import os 
    import csv 
    import numpy as np
    #schools = ["notredame","uofm","columbia","dartmouth","UCSD","berkeley","Harvard","ucla"]
    #s=schools[3]#use this index to control which network we are going to run.
    t="comment"
    
    os.chdir(path)
    name="label_emt3_"+s+"_cm.csv"
    truelabels=pd.read_csv(name,skip_blank_lines=True,header=None)
    name="feature_"+st+".csv"
    truefeatures=pd.read_csv(name,skip_blank_lines=True,header=None).values
    #name="feature_"+s+t+".npz"
    #truefeatures=scipy.sparse.load_npz(name)
    #truefeatures=truefeatures.toarray()

    N=truefeatures.shape[0]
    name="CCsym_"+s+".npz"
    dat_cc=scipy.sparse.load_npz(name);dat_cc=dat_cc.toarray();
    #name="CCasym_"+s+".npz"
    #dat_cca=scipy.sparse.load_npz(name);dat_cca=dat_cca.toarray();
    name="CSCsy_"+s+".npz"
    dat_csc=scipy.sparse.load_npz(name);dat_csc=dat_csc.toarray();
    name="CACsy_"+s+".npz"
    dat_cac=scipy.sparse.load_npz(name);dat_cac=dat_cac.toarray();
    #rownetworks = [dat_csc,dat_cac]#,dat_cc]
    #rownetworks = [np.transpose(dat_cc)]
    rownetworks = [(dat_cc)]
    #rownetworks = [dat_cc,dat_csc]#,dat_cac]
    #rownetworks = [dat_cc,dat_cac]#,dat_csc]
    #rownetworks = [np.zeros((N,N))]

    y=truelabels
    name="train_index_"+s+"_cm.p"
    train_idx = pickle.load(open(name,"rb"));train_idx=np.array(train_idx)
    name="to_be_labeled_index_"+s+"_cm.p"
    val_idx = pickle.load(open(name,"rb"));val_idx=np.array(val_idx)
    name="test_index_"+s+"_cm.p"
    test_idx = pickle.load(open(name,"rb"));test_idx=np.array(test_idx)
    print(len(test_idx))
    #use this line to control testing set
    test_idx=np.array(range(N))
    
    train_mask = sample_mask(train_idx, y.shape[0])
    val_mask = sample_mask(val_idx, y.shape[0])
    test_mask = sample_mask(test_idx, y.shape[0])# ?? why copy three times? First for center node, 2 for each metapath.
    
    y_train = np.zeros(y.shape)
    y_val = np.zeros(y.shape)
    y_test = np.zeros(y.shape)
    y=y.values
    y_train[train_mask, :] = y[train_mask, :]
    y_val[val_mask, :] = y[val_mask, :]
    y_test[test_mask, :] = y[test_mask, :]

    # return selected_idx, selected_idx_2
    print('y_train:{}, y_val:{}, y_test:{}, train_idx:{}, val_idx:{}, test_idx:{}'.format(y_train.shape,
                                                                                          y_val.shape,
                                                                                          y_test.shape,
                                                                                          train_idx.shape,
                                                                                          val_idx.shape,
                                                                                          test_idx.shape))
    truefeatures_list = [truefeatures, truefeatures]#, truefeatures, truefeatures]# ?? why copy three times? First for center node, 2 for each metapath.    
    os.chdir(path_code)
    return rownetworks, truefeatures_list, y_train, y_val, y_test, train_mask, val_mask, test_mask



for i in range(len(schools)):
    adj_list, fea_list, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data_dblp(s=schools[i],st=schools_type[i])
    

    if featype == 'adj':
        fea_list = adj_list



    import scipy.sparse as sp



    nb_nodes = fea_list[0].shape[0]
    ft_size = fea_list[0].shape[1]
    nb_classes = y_train.shape[1]

    # adj = adj.todense()
    
    # features = features[np.newaxis]  # [1, nb_node, ft_size]
    fea_list = [fea[np.newaxis] for fea in fea_list]
    adj_list = [adj[np.newaxis] for adj in adj_list]
    y_train = y_train[np.newaxis]
    y_val = y_val[np.newaxis]
    y_test = y_test[np.newaxis]
    train_mask = train_mask[np.newaxis]
    val_mask = val_mask[np.newaxis]
    test_mask = test_mask[np.newaxis]
    print("s")
    #biases_list = [process.adj_to_bias(adj, [nb_nodes], nhood=1) for adj in adj_list
    #biases_list = [process.adj_to_bias(adj.astype('float') ) for adj in adj_list]
    
    #biases_list = [process.adj_to_bias(adj, [nb_nodes], nhood=1) for adj in adj_list]
    import pickle
    name=schools[i]+'adj.p'
    os.chdir(path_data)
    #pickle.dump(biases_list,open(name,"wb"))
    biases_list=pickle.load(open(name,"rb"))
    
    #For two year separate
    print('build graph...')
    #checkpt_file = 'C:\data\HAN-modified\\tranfer learning\model.ckpt'
    with tf.Graph().as_default():
        with tf.name_scope('input'):
            ftr_in_list = [tf.placeholder(dtype=tf.float32,
                                          shape=(batch_size, nb_nodes, ft_size),
                                          name='ftr_in_{}'.format(i))
                           for i in range(len(fea_list))]
            bias_in_list = [tf.placeholder(dtype=tf.float32,
                                           shape=(batch_size, nb_nodes, nb_nodes),
                                           name='bias_in_{}'.format(i))
                            for i in range(len(biases_list))]
            lbl_in = tf.placeholder(dtype=tf.int32, shape=(
                batch_size, nb_nodes, nb_classes), name='lbl_in')
            msk_in = tf.placeholder(dtype=tf.int32, shape=(batch_size, nb_nodes),
                                    name='msk_in')
            attn_drop = tf.placeholder(dtype=tf.float32, shape=(), name='attn_drop')
            ffd_drop = tf.placeholder(dtype=tf.float32, shape=(), name='ffd_drop')
            is_train = tf.placeholder(dtype=tf.bool, shape=(), name='is_train')
        # forward
        logits, final_embedding, att_val = model.inference(ftr_in_list, nb_classes, nb_nodes, is_train,attn_drop, ffd_drop,bias_mat_list=bias_in_list,hid_units=hid_units, n_heads=n_heads,residual=residual, activation=nonlinearity)
    
        # cal masked_loss
        log_resh = tf.reshape(logits, [-1, nb_classes])
        lab_resh = tf.reshape(lbl_in, [-1, nb_classes])
        msk_resh = tf.reshape(msk_in, [-1])
        loss = model.masked_softmax_cross_entropy(log_resh, lab_resh, msk_resh)
        accuracy = model.masked_accuracy(log_resh, lab_resh, msk_resh)
        # optimzie
        train_op = model.training(loss, lr, l2_coef)
        vlss_mn = np.inf
        vacc_mx = 0.0
        curr_step = 0
        saver = tf.train.Saver()
        init_op = tf.group(tf.global_variables_initializer(),
                           tf.local_variables_initializer())
        #new_saver = tf.train.import_meta_graph('C:\data\HAN-modified\\tranfer learning\model.ckpt.meta')
    
    
        with tf.Session(config=config) as sess:
            #sess.run(init_op)

            train_loss_avg = 0
            train_acc_avg = 0
            val_loss_avg = 0
            val_acc_avg = 0
            os.chdir(path_code)
            saver.restore(sess, path_result+"acm_allMP_multi_fea_.ckpt")
            #saver.restore(sess, "C:\data\HAN-modified\\tranfer learning\\model.ckpt")
            #new_saver.restore(sess, tf.train.latest_checkpoint('./'))
            print("Model restored.")
            
            #saver.restore(sess, checkpt_file)
            #print('load model from : {}'.format(checkpt_file))
            ts_size = fea_list[0].shape[0]
            print(fea_list[0].shape)
            ts_step = 0
            ts_loss = 0.0
            ts_acc = 0.0
            print("test start")
            while ts_step * batch_size < ts_size:
                print(ts_step)
                print(batch_size)
                print(ts_size)
                # fd1 = {ftr_in: features[ts_step * batch_size:(ts_step + 1) * batch_size]}
                fd1 = {i: d[ts_step * batch_size:(ts_step + 1) * batch_size]
                       for i, d in zip(ftr_in_list, fea_list)}
                fd2 = {i: d[ts_step * batch_size:(ts_step + 1) * batch_size]
                       for i, d in zip(bias_in_list, biases_list)}
                fd3 = {lbl_in: y_test[ts_step * batch_size:(ts_step + 1) * batch_size],
                       msk_in: test_mask[ts_step * batch_size:(ts_step + 1) * batch_size],
                
                       is_train: False,
                       attn_drop: 0.0,
                       ffd_drop: 0.0}
            
                fd = fd1
                fd.update(fd2)
                fd.update(fd3)
                
                loss_value_ts, acc_ts, jhy_final_embedding, predicted = sess.run([loss, accuracy, final_embedding, log_resh],
                                                                      feed_dict=fd)
                import pickle
                name=schools[i]+'.p'
                os.chdir(path_result)
                pickle.dump(predicted,open(name,"wb"))
                ts_loss += loss_value_ts
                ts_acc += acc_ts
                ts_step += 1

            print('Test loss:', ts_loss / ts_step,
                  '; Test accuracy:', ts_acc / ts_step)
    
            
            sess.close()


Dataset: acm
----- Opt. hyperparams -----
100
y_train:(1308, 2), y_val:(1308, 2), y_test:(1308, 2), train_idx:(80,), val_idx:(1128,), test_idx:(1308,)
s
build graph...
de
INFO:tensorflow:Restoring parameters from C:\Backup of covid project\2cls_negVSnonneg\results\acm_allMP_multi_fea_.ckpt
Model restored.
(1, 1308, 768)
test start
0
1
1
Test loss: 0.10332329571247101 ; Test accuracy: 0.46712538599967957
100
y_train:(7064, 2), y_val:(7064, 2), y_test:(7064, 2), train_idx:(106,), val_idx:(6858,), test_idx:(7064,)
s
build graph...
de
INFO:tensorflow:Restoring parameters from C:\Backup of covid project\2cls_negVSnonneg\results\acm_allMP_multi_fea_.ckpt
Model restored.
(1, 7064, 768)
test start
0
1
1
Test loss: 0.02360055409371853 ; Test accuracy: 0.672706663608551
100
y_train:(10362, 2), y_val:(10362, 2), y_test:(10362, 2), train_idx:(83,), val_idx:(10179,), test_idx:(10362,)
s
build graph...
de
INFO:tensorflow:Restoring parameters from C:\Backup of covid project\2cls_negVSnonneg\results\a